In [ ]:
/*
使用Declarative的編寫風格來完成與先前imperative編寫風格的相同任務
並同時稍微引入 scala 在實踐fp時，所使用的資料型別。
*/

In [ ]:
/*
相關套件引入。
scalaj-http  // http client 
play-json  // json處理套件
vegas  // 可視化套件
http4s // 輕量 http server套件 
*/
classpath.add(

  "org.scalaj" %% "scalaj-http" % "2.3.0",
  
  "com.typesafe.play" %% "play-json" % "2.6.2",
  
  "org.vegas-viz" %% "vegas" % "0.3.9",
  
  "org.http4s" %% "http4s-dsl" % "0.15.16a",
  "org.http4s" %% "http4s-blaze-server" % "0.15.16a",
  "org.http4s" %% "http4s-blaze-client" % "0.15.16a"
)

In [ ]:
/*
爬取104 與 scala相關的工作
*/

import scalaj.http._
val response: HttpResponse[String] = Http("http://www.104.com.tw/i/apis/jobsearch.cfm")
  .param("kws","java")
  .param("kwop","3")
  .param("role","0")
  .param("fmt","8")
  .param("page","1")
  .param("pgsz","50")
  .asString

In [ ]:

// 轉換成Json作處理
import play.api.libs.json._
val resultJson = Json.parse(response.body)
 
// 找出總筆數
val jobSize = resultJson.\("RECORDCOUNT").as[String].toInt

// 以一頁兩千筆的方式計算總共要訪問幾次，生成一個集合 Seq(1,2,3,...)
val freq = 1 to ((jobSize / 100) +1)


In [ ]:
// 爬取數據
val rawDataArray = freq.map{ page =>

  val pageResponse = Http("http://www.104.com.tw/i/apis/jobsearch.cfm")
  .param("kws","java")
  .param("kwop","3")
  .param("role","0")
  .param("fmt","8")
  .param("page",s"$page")
  .param("pgsz","50")
  .asString

// 將每一個response的body 當作最終處理回傳值  
  pageResponse.body
  
}


In [ ]:
// 整理資料的步驟

// 整理成一條Array內包含多條Array
val step1ExtractData = rawDataArray.map{ responseJsonBody =>
    Json.parse(responseJsonBody).\("data").\\("ADDR_NO_DESCRIPT")
  } 

println(step1ExtractData)

//   .groupBy(element => element )  // 統計裡面的值，並告知用值當作key
//   .mapValues(_.size) // 

// println(jobCountMap)


In [ ]:
// 把所有Array 都壓平在一條Array
val step2FlattenToOneArray = step1ExtractData.flatten
println(step2FlattenToOneArray)

In [ ]:
// 把裡面的值當作key，並把相同值歸併在同一組key中
val step3GroupValueByKey = step2FlattenToOneArray.groupBy(element => element )
println(step3GroupValueByKey)

In [ ]:
// 把key所對應到的records 做處理
// 統計長度，並將長度當作回傳值
val step4toKeyMap = step3GroupValueByKey.mapValues(records => records.size)  
println(step4toKeyMap)

In [ ]:
// 將所有步驟一口氣在一個變數內做完
val jobCountMap = rawDataArray.map{ responseJsonBody =>
    Json.parse(responseJsonBody).\("data").\\("ADDR_NO_DESCRIPT")
  }.flatten.groupBy(element=>element).mapValues(records => records.size) 

In [ ]:
// 將該Map變成一個Seq[tuple(key,value)]，並要求用value做sort排序
val sortJobCountMap = jobCountMap.toSeq.sortBy(  _._2)

// 取前面十筆
val topTen = sortJobCountMap.takeRight(10)

In [ ]:
// 轉成可視化所需的資料格式
val visualTopTen = topTen.map{ record =>
  Map("Country"->record._1, "jobCount" -> record._2)
}
println(visualTopTen)

In [ ]:
// 輸出可視化的html
import vegas._
import vegas.data.External._

val showHtml = Vegas("Java Job Count by each location ")
  .withData( visualTopTen )
  .encodeX("Country", Ordinal)
  .encodeY("jobCount", Quantitative)
  .mark(Bar).html.pageHTML()

println(showHtml)

In [ ]:
/*
將showHtml寫進檔案內
檔案寫入時可能會出意外，我們選擇使用scala的特有container type，類似try catch之功效
*/

val newHtmlFile = new java.io.FileWriter("104-Demo-Declarative.html",false)
import scala.util.Try

val sideEffect = Try{
  newHtmlFile.write(showHtml)
}

println(sideEffect)